In [1]:
import os
import pandas as pd
import numpy as np
import json
import string
import re
from tqdm import tqdm
import pandas as pd
import glob
import pickle

Merge csv

In [14]:
all_files = glob.glob(os.path.join("./result_minie/a*r.csv"))
combined_csv = pd.concat([pd.read_csv(f) for f in all_files ])
combined_csv.to_csv("./result_minie/abstract.csv", index=False)

Remove

In [2]:
df = pd.read_csv("./result_minie/abstract.csv")
sorted_df = df.sort_values("subject")

In [3]:
sorted_df.head()

,subject,relation,object
5757675,#,were elevated in,deceased patients
5757674,#,were,elevated
2762704,#,is,clinicaltrials.gov
379823,# 7131,be compared to,# QUANT_O_1 in usual care group with QUANT_O_2...
50381,# QUANT_S_1,is,QUANT_O_1


In [4]:
def isalnum(text):
    return text.replace(" ", "").replace("-", "").replace("_", "").isalnum()

In [5]:
def isNaN(num):
    return num != num

In [6]:
def same(row1, row2):
    return row1.subject == row2.subject and row1.relation == row2.relation and row1.object == row2.object

In [7]:
last_row = sorted_df.iloc[1]
result = []
for i in tqdm(range(len(sorted_df.index))):
    row = sorted_df.iloc[i]
    # remove NaN
    if not isNaN(row.subject) and not isNaN(row.object):
        # remove special symbol
        if isalnum(row.subject) and isalnum(row.object):
            # remove the entities longer than 3 words
            if len(row.subject.split()) <4 and len(row.object.split()) <4:
                # no single label word subject
                if not (len(row.subject.split()) == 1 and row.subject.startswith("QUANT")):
                    # no single label word object
                    if not (len(row.object.split()) == 1 and row.object.startswith("QUANT")):
                        # different from the previous line
                        if not same(row, last_row):
                            last_row = row
                            result.append(row)

100%|█████████████████████████████████████████████████████████████████████| 6210457/6210457 [09:23<00:00, 11011.65it/s]


In [8]:
# skippable
pickle.dump(result, open('./result_minie/abstract_cleaned.sav', 'wb'))

SAV to CSV

In [2]:
result = pickle.load(open('./result_minie/abstract_cleaned.sav', 'rb'))

In [3]:
results = pd.DataFrame(result)
results.to_csv('minie-relat.csv', index=False)

Create entity

In [2]:
df = pd.read_csv("./minie-relat.csv")

In [3]:
set_1 = set(df['subject'])
set_2 = set(df['object'])
set_entityies = list(set_1) + list(set_2 - set_1)
results = pd.DataFrame(list(set_entityies))
results.columns = ["entity_name"]
sorted_df = results.sort_values("entity_name")
sorted_df['entity_id'] = range(1, len(sorted_df) + 1)

In [4]:
order = ['entity_id','entity_name']
sorted_df = sorted_df[order]

In [5]:
sorted_df

,entity_id,entity_name
381248,1,-RSB-
64389,2,-RSB- apelin-13
237874,3,-RSB- concentrations
949897,4,-RSB- existence
935653,5,-RSB- famous herb
...,...,...
50382,1004383,ﬁndings
387115,1004384,ﬁrst
552753,1004385,ﬁrst paper
939301,1004386,𝒪 memory


In [6]:
sorted_df.to_csv('minie-entity.csv', index=False)

Link entity_id

In [3]:
relation = pd.read_csv("./minie-relat.csv")
entity = pd.read_csv("./minie-entity.csv")

In [3]:
entity_dic = {}
for i in tqdm(range(len(entity.index))):
    row = entity.iloc[i]
    entity_dic[row['entity_name']] = row['entity_id']
relation['subject_id'] = relation.apply(lambda row: entity_dic[row['subject']],axis=1)
relation['object_id'] = relation.apply(lambda row: entity_dic[row['object']],axis=1)
order = ['subject_id','subject','relation','object_id','object']
relation = relation[order]

100%|█████████████████████████████████████████████████████████████████████| 1004387/1004387 [01:10<00:00, 14166.79it/s]


In [11]:
entity.columns

Index(['entity_id', 'entity_name'], dtype='object')

In [10]:
relation.columns

Index(['subject_id', 'subject', 'relation', 'object_id', 'object'], dtype='object')

In [12]:
entity.columns = ['entity_id:ID', 'entity_name']
relation.columns = ['subject_id:START_ID', 'subject', 'relation', 'object_id:END_ID', 'object']

In [4]:
relation['relation_type:TYPE'] = 'minie'

In [14]:
entity.to_csv('minie-entity.csv', index=False)
relation.to_csv('minie-relation.csv', index=False)

## Create neo4j database

```shell
docker run --name kg-minie --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" -p7474:7474 -p7687:7687 -d -v D:/neo4j-minie/data:/data -v D:/neo4j-minie/logs:/logs -v D:/neo4j-minie/import:/var/lib/neo4j/import -v D:/neo4j-minie/plugins:/var/lib/neo4j/plugins -v D:/neo4j-minie/conf:/var/lib/neo4j/conf --env NEO4J_AUTH=neo4j/1234 neo4j
```
> windows

```shell
docker run --name kg-minie -p7474:7474 -p7687:7687 -d -v /home/neo4j-minie/data:/data -v /home/neo4j-minie/logs:/logs -v /home/neo4j-minie/import:/var/lib/neo4j/import -v /home/neo4j-minie/plugins:/var/lib/neo4j/plugins -v /home/neo4j-minie/conf:/var/lib/neo4j/conf --env NEO4J_AUTH=neo4j/1234 neo4j
```
> linux

> 1. import csv, conf
> 2. restart neo4j
> 3. neo4j admin import

neo4j admin import

> Terminal:

```shell
bin/neo4j-admin import --database=kg-minie --nodes=import/minie-entity.csv --relationships=import/minie-relation.csv
```

> Old script backup (Skip)
```cypher
CREATE CONSTRAINT entityIdConstraint ON (ent:Entity) ASSERT ent.entity_id IS UNIQUE
```
```cypher
LOAD CSV WITH HEADERS FROM "file:///minie-entity.csv" AS csvLine
Merge (:Entity {entity_id: toInteger(csvLine.entity_id), entity_name: csvLine.entity_name})
```
```cypher
LOAD CSV WITH HEADERS FROM "file:///minie-relation.csv" AS csvLine
MATCH (s:Entity {entity_id: toInteger(csvLine.subject_id), entity_name:csvLine.subject}), (o:Entity {entity_id: toInteger(csvLine.object_id), entity_name:csvLine.object})
CALL apoc.create.relationship(s, csvLine.relation, {type:'relation_type'}, o)
YIELD rel
RETURN rel
```